In [ ]:
"""
<영화리뷰 긍정/부정 예측하기>
 1. 사용자로부터 영화리뷰 입력 받기
 2. 함수를 이용하여 예측 기능 수행하기
 3. 긍정/부정 출력하기
  -> 출력 예시 : "00% 확률의 [긍정 or 부정] 리뷰 입니다."
  
 4. 참고로, 훈련에서 사용된 말뭉치 tokenizer가 필요합니다.
  - 훈련 시에 훈련에 적용된 tokenizer의 최종본을 파일로 저장해야 합니다.
  - 저장 방법 : pickle 사용하여 저장
"""

In [1]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Input, Dense, Conv2D, MaxPool2D, Dropout, SimpleRNN, Embedding, LSTM, GRU, RepeatVector, TimeDistributed
from keras.models import Sequential
from keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
tf.keras.utils.set_random_seed(42)

### 텍스트 길이 정규화 라이브러리
# - 텍스트의 길이가 긴 경우에는 자르고, 길이가 짧은 경우에는 채움
from tensorflow.keras.preprocessing.sequence import pad_sequences


### 말뭉치 사전 처리를 위한 라이브러리
# - 텍스트 데이터를 수자(인덱스번호)로 변환하는 라이브러리
from tensorflow.keras.preprocessing.text import Tokenizer

In [3]:
import pickle

with open("./files/tokenizer.pkl", "rb") as fr:
    tokenizer = pickle.load(fr)

In [4]:
tokenizer

In [6]:
model = tf.keras.models.load_model("./model/best_LSTM_model.keras")
model

In [18]:
### 형태소 분석
from konlpy.tag import Okt

### 불용어 정의하기
STOPWORDS = [
    "아", "의", "가", "이", "은", "들", "는", "좀", "잘", "걍",
    "과", "도", "를", "으로", "자", "에", "와", "한", "하다", "흠",
    "ㅠㅠ", "ㅋ", "ㅋㅋ", "ㅎ", "ㅎㅎ", "전",
    "그", "저", "더", "또", "에서", "에게", "때문에", "하지만", "그러나", "그리고",
    "뭐", "왜", "어떻게", "어디", "누가", "뭔가", "그냥", "요", "제", "제발",
    "응", "음", "아니", "맞아", "같이", "좀", "정말", "진짜", "이런", "저런",
    "뭐지", "이건", "저건", "그건", "것", "거", "되게", "너무", "많이", "조금",
    "완전", "약간", "그래도", "그렇게", "항상", "계속", "때", "이제", "또한",
    "부터", "까지", "처럼", "만큼", "대해", "대한", "위해"
]


In [19]:
okt = Okt()

In [22]:
def get_answer(user_input):
    tokenized = okt.morphs(user_input)
    filtered = [word for word in tokenized if word not in STOPWORDS]
    input_user = tokenizer.texts_to_sequences([' '.join(filtered)])
    padded_user = pad_sequences(input_user, maxlen=11, padding="post", truncating="post")
    pred = model.predict(padded_user, verbose=0)
    sentiment = "긍정" if pred >= 0.5 else "부정"
    percent = float(pred.item() * 100) if pred.item() >= 0.5 else float((1 - pred.item()) * 100)
    print(f"{percent:.2f}% 확률의 [{sentiment}] 리뷰입니다.")
    return

while True:
    user_input = input("영화 리뷰를 작성해주세요. (종료는 'Q')")
    
    if user_input == None:
        print("입력된 리뷰가 없습니다.")
        
    elif user_input.strip().upper() == 'Q':
        print("서비스를 종료합니다.")
        break
    
    print("사용자 :", user_input)
    answer = get_answer(user_input)

사용자 : 인생에서 꼭 한 번 이상은 무조건 봐야 하는 걸작  왜 좋은 영화인지 2시간 이상 떠들 수 있음  이마루 취향 저격 200%
99.31% 확률의 [긍정] 리뷰입니다.
사용자 : 어디에 내놔도 부족하지 않은 수작 이상의 걸작  영화를 보고 느낄 수 있는 재미의 최고점 경험
99.55% 확률의 [긍정] 리뷰입니다.
사용자 : 영화적인 만족도가 상당히 높은 작품  기억에 오랫동안 남는 작품
92.80% 확률의 [긍정] 리뷰입니다.
사용자 :  완벽하진 못하지만 영화가 가진 힘이 분명한 작품
93.73% 확률의 [긍정] 리뷰입니다.
사용자 : 보고 나서 시간이 아깝지는 않은 영화  장점이 또렷하게 있지만 아쉬움도 남는 평범한 작품  상업 영화 기준 해당 별점 이상은 나름 추천은 하는 편
67.47% 확률의 [긍정] 리뷰입니다.
사용자 : 이 별점부터는 영화를 보고 나서 짜증이 나기도 함
95.84% 확률의 [부정] 리뷰입니다.
사용자 : 졸작, 어디 하나 잘 난 데 없는 질적으로 낮은 작품
94.86% 확률의 [부정] 리뷰입니다.
사용자 : 망작, 이 영화를 보는데 쓴 시간이 아까운 작품
99.41% 확률의 [부정] 리뷰입니다.
사용자 : 영화라는 분류도 붙는 게 과분한 영상 쓰레기
97.81% 확률의 [부정] 리뷰입니다.
서비스를 종료합니다.


In [23]:
def preprocess_text(text):
    tokens = okt.morphs(text)
    print("형태소 분석 결과:", tokens)  # 디버깅용 출력
    filtered = [word for word in tokens if word not in STOPWORDS]
    print("불용어 제거 후:", filtered)  # 디버깅용 출력
    return ' '.join(filtered)

In [24]:
test_sentence = "인생에서 꼭 한 번 이상은 무조건 봐야 하는 걸작  왜 좋은 영화인지 2시간 이상 떠들 수 있음  이마루 취향 저격 200%"
tokens = okt.morphs(test_sentence)
filtered = [word for word in tokens if word not in STOPWORDS]
print("원문 토큰:", tokens)
print("불용어 제거 토큰:", filtered)

원문 토큰: ['인생', '에서', '꼭', '한', '번', '이상은', '무조건', '봐야', '하는', '걸작', '왜', '좋은', '영화', '인지', '2시간', '이상', '떠들', '수', '있음', '이마루', '취향', '저격', '200%']
불용어 제거 토큰: ['인생', '꼭', '번', '이상은', '무조건', '봐야', '하는', '걸작', '좋은', '영화', '인지', '2시간', '이상', '떠들', '수', '있음', '이마루', '취향', '저격', '200%']
